In [8]:
!pip install python-dotenv
# set environment variables
import os
from dotenv import load_dotenv
# self.login = os.getenv('EMAIL_LOGIN')
# self.password = os.getenv('EMAIL_PASSWORD')
# self.smtp_server = os.getenv('SMTP_SERVER')
# self.smtp_port = int(os.getenv('SMTP_PORT', 587))  # Default SMTP port is set to 587

load_dotenv()


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


True

In [16]:
import smtplib
import datetime
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import formatdate
from email import encoders

class EmailCalendarInvite:
    CRLF = "\r\n"

    def __init__(self):
        # Retrieve SMTP configuration from environment variables
        self.login = os.getenv('EMAIL_LOGIN')
        self.password = os.getenv('EMAIL_PASSWORD')
        self.smtp_server = os.getenv('SMTP_SERVER')
        self.smtp_port = int(os.getenv('SMTP_PORT', 587))  # Default SMTP port is set to 587

    def create_invite(self, attendees, subject, body, start, organizer='organizer', description=None, duration=None, end=None):
        if end is None:
            if duration is None:
                duration = datetime.timedelta(minutes=30)
            end = start + duration

        organizer = f"ORGANIZER;CN={organizer}" + self.CRLF
        fro_m = f"{os.getenv('FROM_NAME', 'nickname')} <{self.login}>"

        dtstart = start.strftime("%Y%m%dT%H%M%SZ")
        dtend = end.strftime("%Y%m%dT%H%M%SZ")
        dtstamp = datetime.datetime.now().strftime("%Y%m%dT%H%M%SZ")

        description = "DESCRIPTION: " + description + self.CRLF
        attendee = ""
        for att in attendees:
            attendee += f"ATTENDEE;CUTYPE=INDIVIDUAL;ROLE=REQ-PARTICIPANT;PARTSTAT=ACCEPTED;RSVP=TRUE{self.CRLF};CN={att};X-NUM-GUESTS=0:{self.CRLF}mailto:{att}{self.CRLF}"

        ical = ("BEGIN:VCALENDAR" + self.CRLF + "PRODID:pyICSParser" + self.CRLF + "VERSION:2.0" + self.CRLF + 
                "CALSCALE:GREGORIAN" + self.CRLF + "METHOD:REQUEST" + self.CRLF + "BEGIN:VEVENT" + self.CRLF +
                f"DTSTART:{dtstart}" + self.CRLF + f"DTEND:{dtend}" + self.CRLF + f"DTSTAMP:{dtstamp}" + self.CRLF +
                organizer + self.CRLF + f"UID:FIXMEUID{dtstamp}" + self.CRLF + attendee + "CREATED:" + dtstamp +
                self.CRLF + description + "LAST-MODIFIED:" + dtstamp + self.CRLF + "LOCATION:" + self.CRLF +
                "SEQUENCE:0" + self.CRLF + "STATUS:CONFIRMED" + self.CRLF + "SUMMARY: "+ subject+ self.CRLF + "TRANSP:OPAQUE" + self.CRLF +
                "END:VEVENT" + self.CRLF + "END:VCALENDAR" + self.CRLF)

        msg = MIMEMultipart('mixed')
        msg['Reply-To'] = fro_m
        msg['Date'] = formatdate(localtime=True)
        msg['Subject'] = subject
        msg['From'] = fro_m
        msg['To'] = ",".join(attendees)

        part_email = MIMEText(body, "html")
        part_cal = MIMEText(ical, 'calendar;method=REQUEST')

        msgAlternative = MIMEMultipart('alternative')
        msg.attach(msgAlternative)
        msgAlternative.attach(part_email)
        msgAlternative.attach(part_cal)

        ical_atch = MIMEBase('application/ics', ' ;name="%s"' % ("invite.ics"))
        ical_atch.set_payload(ical)
        encoders.encode_base64(ical_atch)
        ical_atch.add_header('Content-Disposition', 'attachment; filename="%s"' % ("invite.ics"))

        msg.attach(ical_atch)
        return msg

    def send_invite(self, attendees, msg):
        with smtplib.SMTP(self.smtp_server, self.smtp_port) as server:
            server.ehlo()
            server.starttls()
            server.login(self.login, self.password)
            server.sendmail(msg['From'], attendees, msg.as_string())

# Example of using the class

if __name__ == "__main__":
    attendees = []#["leonid.sokolov@big-picture.com"]
    subject = "Frühstücken"
    organizer = "Leonid Sokolov <l.sokolov@gmx.de>"
    body = "Email body visible in the invite of outlook and outlook.com but not google calendar"
    start = datetime.datetime.now() + datetime.timedelta(minutes=25)

    inviter = EmailCalendarInvite()

    attendees.append(input("Enter the email address of the attendee: "))

    msg = inviter.create_invite(attendees, subject, body, start, description="Viel besser so!")
    inviter.send_invite(attendees, msg)
